## FT084 - Introdução a Mineração de Dados
---
### Tarefa 04: Regressão Múltipla  

Este código tem por objetivo a resolução de uma das etapas da tarefa em questão, que consiste na implementação de um algoritmo de regressão múltipla, analisando algumas medidas e selecionando os melhores atributos.  
Instruções para o experimento:
1. Encontrar o melhor conjunto de atributos. Qual é o melhor modelo de acordo com as medidas Cp, BIC, and R2 ajustado? Mostre gráficos para dar suporte a sua resposta e mostre os coeficientes do melhor modelo.
2. Repita o procedimento em (1), usando seleção progressiva e seleção regressiva. Como sua resposta agora se compara aos resultados do item (a)?
3. Realize a seleção de variáveis e de modelos usando validação cruzada no conjunto de treino.
4. Compare os modelos obtidos nos items (1), (2) e (3) no conjunto de teste. Qual deles é o melhor?

---

#### 1) Importação das bibliotecas  
Serão utilizados alguns pacotes para a implementação do código. São eles:
- pandas: leitura dos arquivos
- numpy, scipy: cálculo de algumas estatísticas
- sklearn: modelo de regressão, separação dos dados entre treino e teste, transformação dos atributos categóricos para numéricos (caso necessário), matriz de confusão, avaliação do erro, validação cruzada, seleção de atributos
- statsmodels: modelo de regressão
- matplotly.pyplot e plotly: visualizações extras

In [53]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [21]:
X, y = load_boston(return_X_y=True)
columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = pd.DataFrame(X, columns = columns[0:13]).astype('float64')
target = pd.DataFrame(y, columns = ['MEDV']).astype('float64')
target = target['MEDV']
dataset = pd.merge(data, target, left_index = True, right_index = True)

In [22]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(target,data[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(data[list(feature_set)]) - target) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [23]:
def getBest(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(data.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [24]:
# Could take quite awhile to complete...

models_best = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,13):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed 13 models on 1 predictors in 0.030887126922607422 seconds.
Processed 78 models on 2 predictors in 0.1266613006591797 seconds.
Processed 286 models on 3 predictors in 0.4996638298034668 seconds.
Processed 715 models on 4 predictors in 1.2556424140930176 seconds.
Processed 1287 models on 5 predictors in 2.491337776184082 seconds.
Processed 1716 models on 6 predictors in 3.2941906452178955 seconds.
Processed 1716 models on 7 predictors in 3.3191237449645996 seconds.
Processed 1287 models on 8 predictors in 2.634953498840332 seconds.
Processed 715 models on 9 predictors in 1.5039780139923096 seconds.
Processed 286 models on 10 predictors in 0.557509183883667 seconds.
Processed 78 models on 11 predictors in 0.16954660415649414 seconds.
Processed 13 models on 12 predictors in 0.02792525291442871 seconds.
Total elapsed time: 16.00417137145996 seconds.


In [25]:
models_best

,RSS,model
1,29555.781529,<statsmodels.regression.linear_model.Regressio...
2,15444.934439,<statsmodels.regression.linear_model.Regressio...
3,14343.626020,<statsmodels.regression.linear_model.Regressio...
4,13555.583004,<statsmodels.regression.linear_model.Regressio...
5,13161.006084,<statsmodels.regression.linear_model.Regressio...
6,12895.173642,<statsmodels.regression.linear_model.Regressio...
7,12701.148163,<statsmodels.regression.linear_model.Regressio...
8,12538.094816,<statsmodels.regression.linear_model.Regressio...
9,12439.049646,<statsmodels.regression.linear_model.Regressio...
10,12264.742998,<statsmodels.regression.linear_model.Regressio...


In [ ]:
print(models_best.loc[12, "model"].summary())

In [ ]:
print(getBest(12)["model"].summary())

In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 18, 'lines.markersize': 10})

# Set up a 2x2 grid so we can look at 4 plots at once
plt.subplot(2, 2, 1)

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector
plt.plot(models_best["RSS"])
plt.plot(models_best["RSS"].argmin(), models_best["RSS"].min(), "or")
plt.xlabel('# Predictors')
plt.ylabel('RSS')

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector
rsquared_adj = models_best.apply(lambda row: row[1].rsquared_adj, axis=1)

plt.subplot(2, 2, 2)
plt.plot(rsquared_adj)
plt.plot(rsquared_adj.argmax(), rsquared_adj.max(), "or")
plt.xlabel('# Predictors')
plt.ylabel('adjusted rsquared')

# We'll do the same for AIC and BIC, this time looking for the models with the SMALLEST statistic
aic = models_best.apply(lambda row: row[1].aic, axis=1)
m = len(target)
p = 12
hat_sigma_squared = (1/(m - p -1)) * min(models_best["RSS"])
cp = (1/m) * (models_best["RSS"] + 2 * 506 * hat_sigma_squared )

plt.subplot(2, 2, 3)
plt.plot(cp)
plt.plot(cp.argmin(), cp.min(), "or")
plt.xlabel('# Predictors')
plt.ylabel('Cp')

bic = models_best.apply(lambda row: row[1].bic, axis=1)

plt.subplot(2, 2, 4)
plt.plot(bic)
plt.plot(bic.argmin(), bic.min(), "or")
plt.xlabel('# Predictors')
plt.ylabel('BIC')

In [26]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in data.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [27]:
models_fwd = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
predictors = []

for i in range(1,len(data.columns)+1):    
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  13 models on 1 predictors in 0.023911476135253906 seconds.
Processed  12 models on 2 predictors in 0.019946813583374023 seconds.
Processed  11 models on 3 predictors in 0.018949270248413086 seconds.
Processed  10 models on 4 predictors in 0.016954898834228516 seconds.
Processed  9 models on 5 predictors in 0.015957355499267578 seconds.
Processed  8 models on 6 predictors in 0.013962745666503906 seconds.
Processed  7 models on 7 predictors in 0.012964963912963867 seconds.
Processed  6 models on 8 predictors in 0.010970592498779297 seconds.
Processed  5 models on 9 predictors in 0.00997304916381836 seconds.
Processed  4 models on 10 predictors in 0.006981372833251953 seconds.
Processed  3 models on 11 predictors in 0.0069811344146728516 seconds.
Processed  2 models on 12 predictors in 0.003989458084106445 seconds.
Processed  1 models on 13 predictors in 0.002991914749145508 seconds.
Total elapsed time: 0.19046616554260254 seconds.


In [28]:
print(models_fwd.loc[1, "model"].summary())
print(models_fwd.loc[2, "model"].summary())

                                 OLS Regression Results                                
Dep. Variable:                   MEDV   R-squared (uncentered):                   0.901
Model:                            OLS   Adj. R-squared (uncentered):              0.901
Method:                 Least Squares   F-statistic:                              4615.
Date:                Tue, 11 May 2021   Prob (F-statistic):                   3.74e-256
Time:                        21:17:50   Log-Likelihood:                         -1747.1
No. Observations:                 506   AIC:                                      3496.
Df Residuals:                     505   BIC:                                      3500.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [29]:
def backward(predictors):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [30]:
models_bwd = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(data.columns)))

tic = time.time()
predictors = data.columns

while(len(predictors) > 1):  
    models_bwd.loc[len(predictors)-1] = backward(predictors)
    predictors = models_bwd.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  13 models on 12 predictors in 0.029920101165771484 seconds.
Processed  12 models on 11 predictors in 0.02194070816040039 seconds.
Processed  11 models on 10 predictors in 0.019946575164794922 seconds.
Processed  10 models on 9 predictors in 0.01795196533203125 seconds.
Processed  9 models on 8 predictors in 0.016954898834228516 seconds.
Processed  8 models on 7 predictors in 0.01595783233642578 seconds.
Processed  7 models on 6 predictors in 0.014959335327148438 seconds.
Processed  6 models on 5 predictors in 0.009973526000976562 seconds.
Processed  5 models on 4 predictors in 0.008975982666015625 seconds.
Processed  4 models on 3 predictors in 0.007978439331054688 seconds.
Processed  3 models on 2 predictors in 0.005984067916870117 seconds.
Processed  2 models on 1 predictors in 0.003989458084106445 seconds.
Total elapsed time: 0.1795189380645752 seconds.


In [31]:
print("------------")
print("Best Subset:")
print("------------")
print(models_best.loc[7, "model"].params)

------------
Best Subset:
------------
ZN         0.038211
CHAS       2.971853
RM         5.592302
DIS       -0.682234
PTRATIO   -0.534289
B          0.015695
LSTAT     -0.509523
dtype: float64


In [32]:
print("-----------------")
print("Foward Selection:")
print("-----------------")
print(models_fwd.loc[7, "model"].params)

-----------------
Foward Selection:
-----------------
RM         5.592302
LSTAT     -0.509523
PTRATIO   -0.534289
B          0.015695
DIS       -0.682234
CHAS       2.971853
ZN         0.038211
dtype: float64


In [33]:
print("-------------------")
print("Backward Selection:")
print("-------------------")
print(models_bwd.loc[7, "model"].params)

-------------------
Backward Selection:
-------------------
ZN         0.038211
CHAS       2.971853
RM         5.592302
DIS       -0.682234
PTRATIO   -0.534289
B          0.015695
LSTAT     -0.509523
dtype: float64


In [37]:
model = LinearRegression()
model.fit(data, target)

LinearRegression()

In [39]:
selector = RFECV(model, step=1, cv=12)
selector = selector.fit(data,target)
selector.support_

array([False, False, False,  True,  True,  True, False,  True, False,
       False,  True, False,  True])

In [52]:
cv_results = cross_validate(model, data, target, return_estimator = True)
cv_results

{'fit_time': array([0.00299215, 0.00199461, 0.00199461, 0.00199461, 0.00199461]),
 'score_time': array([0.0009973 , 0.00199485, 0.00099754, 0.00199437, 0.00099707]),
 'estimator': (LinearRegression(),
  LinearRegression(),
  LinearRegression(),
  LinearRegression(),
  LinearRegression()),
 'test_score': array([ 0.63919994,  0.71386698,  0.58702344,  0.07923081, -0.25294154])}